In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key for gmaps.
from config import g_key

In [2]:
# Read the WeatherPy_vacation.csv and store into new DataFrame.
city_data_df = pd.read_csv("challenge_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,0,Yellowknife,CA,2020-05-08 18:37:14,62.46,-114.35,41.00,74,90,13.87,overcast clouds,0,0
1,1,Busselton,AU,2020-05-08 18:36:47,-33.65,115.33,46.99,66,1,6.04,clear sky,0,0
2,2,Los Llanos De Aridane,ES,2020-05-08 18:40:39,28.66,-17.92,71.60,68,75,5.82,broken clouds,0,0
3,3,Boguchany,RU,2020-05-08 18:44:37,58.38,97.45,39.00,77,100,1.30,overcast clouds,0,0
4,4,Kapaa,US,2020-05-08 18:36:11,22.08,-159.32,75.20,69,1,16.11,clear sky,0,0


In [3]:
# Ask the customer to add a minimum and maximum temperature value and if they want precipitation.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain = str(input("Do you want it to be raining? (yes/no) "))
snow = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [4]:
# If statment for filtering thorugh precipitation.
if rain == "yes":
    rain_request = city_data_df["Rain Inches (last 3 hrs)"] != 0
else:
    rain_request = city_data_df["Rain Inches (last 3 hrs)"] == 0
    
if snow == "yes":
    snow_request = city_data_df["Snow Inches (last 3 hrs)"] != 0
else:
    snow_request = city_data_df["Snow Inches (last 3 hrs)"] == 0
    
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp) & 
                                       (rain_request) & (snow_request)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
4,4,Kapaa,US,2020-05-08 18:36:11,22.08,-159.32,75.20,69,1,16.11,clear sky,0,0
23,23,Vaini,TO,2020-05-08 18:36:27,-21.20,-175.20,78.80,94,75,10.29,broken clouds,0,0
28,28,Tual,ID,2020-05-08 18:44:46,-5.67,132.75,82.89,78,0,19.01,clear sky,0,0
30,30,Kailua,US,2020-05-08 18:44:47,21.40,-157.74,78.80,54,20,13.87,few clouds,0,0
32,32,Butaritari,KI,2020-05-08 18:36:31,3.07,172.79,82.42,75,100,3.49,overcast clouds,0,0


In [5]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [6]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [7]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kapaa,US,75.20,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
23,Vaini,TO,78.80,broken clouds,-21.20,-175.20,Keleti Beach Resort
28,Tual,ID,82.89,clear sky,-5.67,132.75,Grand Vilia Hotel
30,Kailua,US,78.80,few clouds,21.40,-157.74,Sheffield House Bed & Breakfast
32,Butaritari,KI,82.42,overcast clouds,3.07,172.79,Isles Sunset Lodge


In [8]:
# Configure gmaps to use your Google API key.
gmaps.configure(g_key)

In [9]:
# Create info boxes for hotels
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [13]:
# Add a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Create the output file (CSV).
output_data_file = "challenge_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")